In [1]:
import os
import h5py
import numpy as np

input_datafile = h5py.File('../input/uploads/tryph5')

train_norm_images = np.array(input_datafile['train_norm'])
train_trypo_images = np.array(input_datafile['train_trypo'])
valid_norm_images = np.array(input_datafile['valid_norm'])
valid_trypo_images = np.array(input_datafile['valid_trypo'])


In [2]:
print(train_norm_images.shape, train_trypo_images.shape)
print(valid_norm_images.shape, valid_trypo_images.shape)

(10068, 224, 224, 3) (5816, 224, 224, 3)
(500, 224, 224, 3) (500, 224, 224, 3)


In [4]:
max_size = 224

def process_images(data):
    return np.divide(data, max_size)

train_norm_images = process_images(train_norm_images)
train_trypo_images = process_images(train_trypo_images)
valid_norm_images = process_images(valid_norm_images)
valid_trypo_images = process_images(valid_trypo_images)

In [5]:
from keras.utils import np_utils

def get_data(trypo, norm):
    y_trypo = np.ones(len(trypo))
    y_norm = np.zeros(len(norm))
    y = np.concatenate([y_trypo, y_norm])
    y = np_utils.to_categorical(y)
    x = np.concatenate([trypo, norm])
    return x, y

X_train, y_train = get_data(train_norm_images, train_trypo_images)
X_val, y_val = get_data(valid_norm_images, valid_trypo_images)

Using TensorFlow backend.


In [6]:
print(X_train.shape, y_train.shape)
print(X_val.shape, y_val.shape)

(15884, 224, 224, 3) (15884, 2)
(1000, 224, 224, 3) (1000, 2)


In [136]:
import keras
from keras.models import Model
from keras.layers import GlobalAveragePooling2D, Dense, Conv2D
from keras.applications.vgg16 import VGG16


def get_model(layers_cut):
    model = VGG16(include_top = False, weights='imagenet')
    x = model.layers[-layers_cut].output
    layer_size = int(x.get_shape()[-1])
    x = Conv2D(layer_size, 1)(x)
    x = GlobalAveragePooling2D()(x)
    x = Dense(layer_size, activation='relu')(x)
    x = Dense(2, activation = 'softmax')(x)
    model = Model(model.input, x)
    print(model.summary())

    trainable_layers = 4

    for layer in model.layers[:-trainable_layers]:
        layer.trainable = False
    model.compile(optimizer = "adam", loss = "categorical_crossentropy",
                 metrics = ["accuracy"])
    return model


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_100 (InputLayer)       (None, None, None, 3)     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, None, None, 64)    4160      
_________________________________________________________________
global_average_pooling2d_60  (None, 64)                0         
_________________________________________________________________
dense_64 (Dense)             (None, 64)                4160      
__________

In [163]:
#models = [get_model(16), get_model(13), get_model(9), get_model(5)]
model = get_model(16)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_147 (InputLayer)       (None, None, None, 3)     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
conv2d_57 (Conv2D)           (None, None, None, 64)    4160      
_________________________________________________________________
global_average_pooling2d_107 (None, 64)                0         
_________________________________________________________________
dense_158 (Dense)            (None, 64)                4160      
__________

In [161]:
import matplotlib.pyplot as plt

def plot_training_graph(history):
    epochs = len(history['acc'])
    epochs_range = range(1, epochs + 1)
    acc, = plt.plot(epochs_range, history['val_acc'], label='accuracy')
    plt.legend([acc], ['accuracy'])
    plt.show()

In [162]:
results = []

for model in models:
    hist = model.fit(X_train, y_train, validation_data=(X_val, y_val),
                 epochs=10)
    results.append(hist)
    plot_training_graph(hist.history)

Train on 15884 samples, validate on 1000 samples
Epoch 1/10
15884/15884 [==============================] - 96s - loss: 0.5672 - acc: 0.7033 - val_loss: 0.5302 - val_acc: 0.7280
Epoch 2/10
15884/15884 [==============================] - 95s - loss: 0.5086 - acc: 0.7419 - val_loss: 0.5145 - val_acc: 0.7460
Epoch 3/10
11776/15884 [=====================>........] - ETA: 23s - loss: 0.4961 - acc: 0.7515

KeyboardInterrupt: 